In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
import cv2
import yfinance as yf
from datetime import datetime, timedelta
from ultralytics import YOLO
import random
import shutil
from tqdm import tqdm
import yaml
import torch

## 2. Preparing the dataset

### 2.1 Generating candlestick chart images

In [2]:
def generate_candlestick_chart(ticker, period="2y", interval="1d", output_dir="charts"):
    """
    Generate candlestick chart images from financial data
    
    Args:
        ticker (str): Stock ticker symbol
        period (str): Data period (e.g., '2y' for 2 years)
        interval (str): Data interval (e.g., '1d' for daily)
        output_dir (str): Directory to save generated charts
    
    Returns:
        str: Path to the saved chart image
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Fetch data using yfinance
    data = yf.download(ticker, period=period, interval=interval)
    data.reset_index(inplace=True, drop=True)
    data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    
    # Check if data is empty
    if len(data) == 0:
        print(f"No data found for {ticker}")
        return None
    
    # Create figure and axis
    fig, ax = plt.figure(figsize=(12, 8), dpi=100), plt.subplot(1, 1, 1)
    
    # Format date axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    
    # Plot candlestick chart
    width = 0.6
    width2 = 0.05
    
    # Loop through data and plot each candlestick
    for i, (idx, row) in enumerate(data.iterrows()):
        # Calculate colors based on price movement
        if row['Close'] >= row['Open']:
            color = 'green'
            body_height = row['Close'] - row['Open']
        else:
            color = 'red'
            body_height = row['Open'] - row['Close']
        
        # Plot the candlestick body
        rect = Rectangle(
            xy=(i-width/2, min(row['Open'], row['Close'])),
            width=width,
            height=body_height,
            facecolor=color,
            edgecolor='black',
            linewidth=0.5
        )
        ax.add_patch(rect)
        
        # Plot the upper and lower wicks
        ax.plot([i, i], [row['Low'], min(row['Open'], row['Close'])], color='black', linewidth=0.5)
        ax.plot([i, i], [max(row['Open'], row['Close']), row['High']], color='black', linewidth=0.5)
    
    # Set title and labels
    ax.set_title(f"{ticker} Candlestick Chart", fontsize=12)
    ax.set_xlabel("Date", fontsize=10)
    ax.set_ylabel("Price", fontsize=10)
    
    # Adjust the x-axis to show the most recent time period
    ax.set_xlim(-1, len(data) + 1)
    
    # Set y-axis limits with some padding
    y_min = data['Low'].min() * 0.95
    y_max = data['High'].max() * 1.05
    ax.set_ylim(y_min, y_max)
    
    # Generate a unique filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = os.path.join(output_dir, f"{ticker}_{timestamp}.jpg")
    
    # Save the chart as an image
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close(fig)
    
    return output_path

### 2.2 Generating candlestick patterns

In [ ]:
from generate_candlestick_data import generate_pattern_dataset

### 2.4 Create the dataset

In [5]:
from generate_candlestick_data import detect_head_and_shoulders_bottom, detect_head_and_shoulders_top, detect_m_head, detect_stock_line, detect_triangle, detect_w_bottom
def create_dataset(output_dir="candlestick_dataset"):
    """
    Create a complete dataset for training YOLOv8
    
    Args:
        output_dir (str): Directory to save the dataset
    """
    # Define stock tickers to use (major indices, tech stocks, etc.)
    tickers = [
        "SPY", "QQQ", "DIA", "AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", 
        "NVDA", "AMD", "INTC", "JPM", "V", "MA", "DIS", "NFLX", "CSCO", "VZ",
        "T", "PFE", "MRK", "JNJ", "PG", "KO", "PEP", "WMT", "HD", "BA", "CAT"
    ]
    
    # Define patterns to detect
    patterns = {
        "head_and_shoulders_bottom": detect_head_and_shoulders_bottom,
        "head_and_shoulders_top": detect_head_and_shoulders_top,
        "m_head": detect_m_head,
        "stock_line": detect_stock_line,
        "triangle": detect_triangle,
        "w_bottom": detect_w_bottom
    }
    
    # Generate pattern dataset
    print("Generating pattern dataset...")
    chart_paths = generate_pattern_dataset(
        tickers=tickers,
        patterns=patterns,
        num_samples_per_pattern=300,  # 300 samples per pattern
        output_dir=os.path.join(output_dir, "raw_data")
    )
    
    # Check if any charts were generated
    if len(chart_paths) == 0:
        print("No pattern charts were generated. Please check the pattern detection logic.")
        return None
    
    # Split into train, val, test sets
    print(f"Generated {len(chart_paths)} chart images")
    
    # Shuffle paths
    random.shuffle(chart_paths)
    
    # Split: 70% train, 20% val, 10% test
    train_split = int(0.7 * len(chart_paths))
    val_split = int(0.9 * len(chart_paths))
    
    train_paths = chart_paths[:train_split]
    val_paths = chart_paths[train_split:val_split]
    test_paths = chart_paths[val_split:]
    
    # Create directories
    os.makedirs(os.path.join(output_dir, "train", "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "train", "labels"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "val", "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "val", "labels"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "test", "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "test", "labels"), exist_ok=True)
    
    # Copy files to appropriate directories
    def copy_files(paths, dest_type):
        for img_path in tqdm(paths, desc=f"Copying {dest_type} files"):
            # Copy image
            img_filename = os.path.basename(img_path)
            shutil.copy(
                img_path,
                os.path.join(output_dir, dest_type, "images", img_filename)
            )
            
            # Copy annotation
            label_path = img_path.replace(".jpg", ".txt")
            label_filename = os.path.basename(label_path)
            shutil.copy(
                label_path,
                os.path.join(output_dir, dest_type, "labels", label_filename)
            )
    
    print("Organizing dataset...")
    copy_files(train_paths, "train")
    copy_files(val_paths, "val")
    copy_files(test_paths, "test")
    
    # Create dataset YAML file
    dataset_yaml = {
        "path": os.path.abspath(output_dir),
        "train": "train/images",
        "val": "val/images",
        "test": "test/images",
        "names": list(patterns.keys())
    }
    
    with open(os.path.join(output_dir, "dataset.yaml"), "w") as f:
        yaml.dump(dataset_yaml, f)
    
    print(f"Dataset created at {output_dir}")
    print(f"Training set: {len(train_paths)} images")
    print(f"Validation set: {len(val_paths)} images")
    print(f"Test set: {len(test_paths)} images")
    
    return os.path.join(output_dir, "dataset.yaml")

## 3. Training the YOLOv8 model

In [6]:
def train_yolov8_model(dataset_yaml, model_size="n", epochs=100, batch_size=16, image_size=640):
    """
    Train a YOLOv8 model on the candlestick pattern dataset
    
    Args:
        dataset_yaml (str): Path to dataset YAML file
        model_size (str): YOLOv8 model size (n, s, m, l, x)
        epochs (int): Number of training epochs
        batch_size (int): Batch size
        image_size (int): Image size
        
    Returns:
        str: Path to the trained model weights
    """
    print(f"Training YOLOv8{model_size} model on {dataset_yaml}...")
    
    # Load a pre-trained YOLOv8 model
    model = YOLO(f"yolov8{model_size}.pt")
    
    # Train the model
    results = model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=image_size,
        patience=20,  # Early stopping patience
        save=True,
        device="0" if torch.cuda.is_available() else "cpu",
        verbose=True
    )
    
    # Get path to best model weights
#     best_weights_path = model.best
    
#     print(f"Training completed. Best weights saved to: {best_weights_path}")
    
    return model

## 4. Fine-tuning the model

In [7]:
def fine_tune_model(best_weights_path, dataset_yaml, epochs=50, batch_size=8, image_size=640):
    """
    Fine-tune the YOLOv8 model
    
    Args:
        best_weights_path (str): Path to the best weights from initial training
        dataset_yaml (str): Path to dataset YAML file
        epochs (int): Number of fine-tuning epochs
        batch_size (int): Batch size
        image_size (int): Image size
        
    Returns:
        str: Path to the fine-tuned model weights
    """
    print(f"Fine-tuning model {best_weights_path}...")
    
    # Load the trained model
    model = YOLO(best_weights_path)
    
    # Fine-tune with a lower learning rate
    results = model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=image_size,
        patience=20,  # Early stopping patience
        save=True,
        device="0" if torch.cuda.is_available() else "cpu",
        verbose=True,
        lr0=0.001,  # Lower learning rate for fine-tuning
        lrf=0.01,   # Final learning rate as a fraction of initial lr
    )
    
    # Get path to best fine-tuned weights
#     best_weights_path = model.best
    
#     print(f"Fine-tuning completed. Best weights saved to: {best_weights_path}")
    
    return model

## 5. Inference for pattern detection

In [8]:
def detect_patterns(chart_image_path, model_path, conf_threshold=0.25):
    """
    Detect candlestick patterns in a chart image
    
    Args:
        chart_image_path (str): Path to the chart image
        model_path (str): Path to the trained YOLOv8 model weights
        conf_threshold (float): Confidence threshold for detections
        
    Returns:
        list: List of detected patterns with confidence scores
    """
    # Load the model
    model = YOLO(model_path)
    
    # Run inference
    results = model(chart_image_path, conf=conf_threshold)
    
    # Process results
    detections = []
    
    for result in results:
        for i, (box, conf, cls) in enumerate(zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls)):
            x1, y1, x2, y2 = box.tolist()
            confidence = conf.item()
            class_id = int(cls.item())
            class_name = result.names[class_id]
            
            detections.append({
                "pattern": class_name,
                "confidence": confidence,
                "box": [x1, y1, x2, y2]
            })
    
    return detections

def visualize_detections(chart_image_path, detections, output_path=None):
    """
    Visualize detected patterns on a chart image
    
    Args:
        chart_image_path (str): Path to the chart image
        detections (list): List of detected patterns
        output_path (str, optional): Path to save the output image
        
    Returns:
        numpy.ndarray: Image with visualized detections
    """
    # Load the image
    image = cv2.imread(chart_image_path)
    
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Colors for different patterns (in RGB)
    colors = {
        "doji": (255, 0, 0),          # Red
        "hammer": (0, 255, 0),        # Green
        "shooting_star": (0, 0, 255), # Blue
        "engulfing": (255, 255, 0),   # Yellow
        "morning_star": (255, 0, 255),# Magenta
        "three_white_soldiers": (0, 255, 255) # Cyan
    }
    
    # Draw bounding boxes and labels
    for detection in detections:
        pattern = detection["pattern"]
        conf = detection["confidence"]
        box = detection["box"]
        
        # Get color for this pattern
        color = colors.get(pattern, (200, 200, 200))
        
        # Draw bounding box
        cv2.rectangle(
            image, 
            (int(box[0]), int(box[1])), 
            (int(box[2]), int(box[3])), 
            color, 
            2
        )
        
        # Prepare label text
        label = f"{pattern}: {conf:.2f}"
        
        # Get text size
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
        
        # Calculate text background rectangle
        text_bg_x1 = int(box[0])
        text_bg_y1 = int(box[1]) - text_size[1] - 10
        text_bg_x2 = int(box[0]) + text_size[0] + 10
        text_bg_y2 = int(box[1])
        
        # Draw text background
        cv2.rectangle(
            image,
            (text_bg_x1, text_bg_y1),
            (text_bg_x2, text_bg_y2),
            color,
            -1
        )
        
        # Draw text
        cv2.putText(
            image,
            label,
            (int(box[0] + 5), int(box[1] - 5)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 255),
            2
        )
    
    # Save the output image if requested
    if output_path:
        # Convert RGB back to BGR for saving
        output_img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, output_img)
    
    return image

def detect_patterns_from_ticker(ticker, model_path, period="1y", interval="1d", conf_threshold=0.3):
    """
    Generate a chart for a ticker and detect patterns
    
    Args:
        ticker (str): Stock ticker symbol
        model_path (str): Path to trained model
        period (str): Data period
        interval (str): Data interval
        conf_threshold (float): Detection confidence threshold
        
    Returns:
        tuple: (chart_path, detections, visualization)
    """
    # Generate chart
    chart_path = generate_candlestick_chart(
        ticker=ticker,
        period=period,
        interval=interval,
        output_dir="inference_charts"
    )
    
    if chart_path is None:
        return None, [], None
    
    # Detect patterns
    detections = detect_patterns(
        chart_image_path=chart_path,
        model_path=model_path,
        conf_threshold=conf_threshold
    )
    
    # Visualize detections
    output_path = chart_path.replace(".jpg", "_detected.jpg")
    visualization = visualize_detections(
        chart_image_path=chart_path,
        detections=detections,
        output_path=output_path
    )
    
    return chart_path, detections, visualization

## 6. Main execution flow

In [9]:
# 1. Create dataset
dataset_yaml = create_dataset(output_dir="candlestick_dataset")

# Check if dataset was created successfully
if dataset_yaml is None:
    print("Dataset creation failed. Please check the pattern detection functions.")

Generating pattern dataset...


Processing tickers:   0%|          | 0/30 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1257 entries, 2020-04-21 to 2025-04-21
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1257 non-null   float64
 1   (High, SPY)    1257 non-null   float64
 2   (Low, SPY)     1257 non-null   float64
 3   (Open, SPY)    1257 non-null   float64
 4   (Volume, SPY)  1257 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 58.9 KB
None MultiIndex([( 'Close', 'SPY'),
            (  'High', 'SPY'),
            (   'Low', 'SPY'),
            (  'Open', 'SPY'),
            ('Volume', 'SPY')],
           names=['Price', 'Ticker'])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1257 non-null   float64
 1   High    1257 non-null   float64
 2   Low     1257 non-null   float64
 3   Open    1257 non-null 

...

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1257 entries, 2020-04-21 to 2025-04-21
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, CAT)   1257 non-null   float64
 1   (High, CAT)    1257 non-null   float64
 2   (Low, CAT)     1257 non-null   float64
 3   (Open, CAT)    1257 non-null   float64
 4   (Volume, CAT)  1257 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 58.9 KB
None MultiIndex([( 'Close', 'CAT'),
            (  'High', 'CAT'),
            (   'Low', 'CAT'),
            (  'Open', 'CAT'),
            ('Volume', 'CAT')],
           names=['Price', 'Ticker'])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1257 non-null   float64
 1   High    1257 non-null   float64
 2   Low     1257 non-null   float64
 3   Open    1257 non-null 

Processing tickers: 100%|██████████| 30/30 [03:59<00:00,  7.97s/it]


Generated chart for three_white_soldiers pattern in CAT

Pattern detection summary:
doji: 3912
hammer: 957
shooting_star: 815
engulfing: 1314
morning_star: 171
three_white_soldiers: 338
Generated 1628 chart images
Organizing dataset...


Copying test files: 100%|██████████| 163/163 [00:01<00:00, 113.82it/s]

Dataset created at candlestick_dataset
Training set: 1139 images
Validation set: 326 images
Test set: 163 images


In [10]:
# 2. Train YOLOv8 model
model_returned = train_yolov8_model(
    dataset_yaml=dataset_yaml,
    model_size="n",  # Start with a smaller model for faster training
    epochs=25,
    batch_size=16,
    image_size=640
)

Training YOLOv8n model on candlestick_dataset/dataset.yaml...


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.2MB/s]


Ultralytics 8.3.111 🚀 Python-3.8.8 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB, 81050MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=candlestick_dataset/dataset.yaml, epochs=25, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 5.35M/5.35M [00:00<00:00, 69.8MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 126.0±11.3 MB/s, size: 47.9 KB)


...

      25/25      2.25G          0     0.1446          0          0        640: 100%|██████████| 72/72 [00:03<00:00, 20.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:00<00:00, 24.90it/s]

                   all        325        325          0          0          0          0



25 epochs completed in 0.031 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.8.8 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB, 81050MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:00<00:00, 17.18it/s]


                   all        325        325          0          0          0          0
                  doji         57         57          0          0          0          0
                hammer         64         64          0          0          0          0
         shooting_star         56         56          0          0          0          0
             engulfing         57         57          0          0          0          0
          morning_star         29         29          0          0          0          0
  three_white_soldiers         62         62          0          0          0          0
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train


In [11]:
# 3. Fine-tune the model
fine_tuned_model = fine_tune_model(
    best_weights_path="runs/detect/train/weights/best.pt",
    dataset_yaml=dataset_yaml,
    epochs=10,
    batch_size=8,
    image_size=640
)

Fine-tuning model runs/detect/train/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.8.8 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB, 81050MiB)
engine/trainer: task=detect, mode=train, model=runs/detect/train/weights/best.pt, data=candlestick_dataset/dataset.yaml, epochs=10, time=None, patience=20, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/candlestick_dataset/train/labels.cache... 1139 images, 0 backgrounds, 2 corrupt: 100%|██████████| 1139/1139 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/candlestick_dataset/train/images/NFLX_engulfing_20250421_145648_7978.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1259]
train: WARNING ⚠️ /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/candlestick_dataset/train/images/NFLX_morning_star_20250421_145650_7318.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.1527]


val: Fast image access ✅ (ping: 0.2±0.0 ms, read: 126.9±14.1 MB/s, size: 50.5 KB)


val: Scanning /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/candlestick_dataset/val/labels.cache... 326 images, 0 backgrounds, 1 corrupt: 100%|██████████| 326/326 [00:00<?, ?it/s]

...

      10/10      1.24G          0   0.001137          0          0        640: 100%|██████████| 143/143 [00:05<00:00, 25.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:00<00:00, 37.32it/s]

                   all        325        325          0          0          0          0



10 epochs completed in 0.018 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.8.8 torch-2.2.2+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB, 81050MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:00<00:00, 35.35it/s]


                   all        325        325          0          0          0          0
                  doji         57         57          0          0          0          0
                hammer         64         64          0          0          0          0
         shooting_star         56         56          0          0          0          0
             engulfing         57         57          0          0          0          0
          morning_star         29         29          0          0          0          0
  three_white_soldiers         62         62          0          0          0          0
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train2


In [12]:
fine_tuned_weights = "runs/detect/train2/weights/best.pt"

In [13]:
 # 4. Test inference on new data
test_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

print("\nTesting pattern detection on new charts:")
for ticker in test_tickers:
    print(f"\nAnalyzing {ticker}...")
    chart_path, detections, viz = detect_patterns_from_ticker(
        ticker=ticker,
        model_path=fine_tuned_weights,
        period="2y",
        interval="1d",
        conf_threshold=0.3
    )
    
    if chart_path is None:
        print(f"Failed to generate chart for {ticker}")
        continue
    
    print(f"Chart saved to: {chart_path}")
    
    if len(detections) > 0:
        print(f"Detected {len(detections)} patterns:")
        for i, det in enumerate(detections):
            print(f"  {i+1}. {det['pattern']} (confidence: {det['confidence']:.2f})")
    else:
        print("No patterns detected")

print("\nModel training and testing completed!")
print(f"Trained model path: {fine_tuned_weights}")


Testing pattern detection on new charts:

Analyzing AAPL...


[*********************100%***********************]  1 of 1 completed



image 1/1 /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/inference_charts/AAPL_20250421_150237.jpg: 448x640 (no detections), 62.9ms
Speed: 2.0ms preprocess, 62.9ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)
Chart saved to: inference_charts/AAPL_20250421_150237.jpg
No patterns detected

Analyzing MSFT...


[*********************100%***********************]  1 of 1 completed



image 1/1 /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/inference_charts/MSFT_20250421_150238.jpg: 448x640 (no detections), 6.3ms
Speed: 1.9ms preprocess, 6.3ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)


...

Chart saved to: inference_charts/GOOGL_20250421_150240.jpg
No patterns detected

Analyzing AMZN...



image 1/1 /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/inference_charts/AMZN_20250421_150241.jpg: 448x640 (no detections), 6.5ms
Speed: 2.4ms preprocess, 6.5ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)


[*********************100%***********************]  1 of 1 completed

Chart saved to: inference_charts/AMZN_20250421_150241.jpg
No patterns detected

Analyzing TSLA...



image 1/1 /home/bajariya.k/cs5330-project-candlestick-pattern-detection/src/inference_charts/TSLA_20250421_150243.jpg: 448x640 (no detections), 6.5ms
Speed: 2.3ms preprocess, 6.5ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)
Chart saved to: inference_charts/TSLA_20250421_150243.jpg
No patterns detected

Model training and testing completed!
Trained model path: runs/detect/train2/weights/best.pt
